In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve,
                                    GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix
from sklearn.metrics.scorer import roc_auc_scorer
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #620
  (fname, cnt))


In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
from iraps_classifier import IRAPSCore, IRAPSClassifier

## drug  (5Z)-7-Oxozeaenol

In [5]:
rna = pd.read_csv('./drug_respond/(5Z)-7-Oxozeaenol.csv', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP11-309M23.1 (ENSGR0000237531),AMDP1 (ENSGR0000237801),BX649553.1 (ENSGR0000263835),BX649553.2 (ENSGR0000263980),BX649553.3 (ENSGR0000264510),BX649553.4 (ENSGR0000264819),RN7SL355P (ENSGR0000265350),MIR3690 (ENSGR0000265658),AL732314.1 (ENSGR0000266731),(5Z)-7-Oxozeaenol
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.862564
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.759749
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.658579
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.821017
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.710453
5,5.943218,0.000000,6.501280,1.937344,4.078951,0.000000,0.432959,5.942280,4.527946,4.472488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.662955
6,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.578093
7,3.438293,0.000000,6.313790,2.286881,4.152995,0.000000,1.691534,6.444270,4.759156,4.106013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.662305
8,2.980025,0.000000,7.118630,1.903038,4.533563,0.000000,3.090853,6.827565,3.969933,3.872829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.699442
9,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.574803


In [6]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
y

0      0.862564
1      0.759749
2      0.658579
3      0.821017
4      0.710453
5      0.662955
6      0.578093
7      0.662305
8      0.699442
9      0.574803
10     0.634965
11     0.664182
12     0.406618
13     0.586659
14     0.413623
15     0.677848
16     0.647090
17     0.042457
18     0.625574
19     0.738274
20     0.469464
21     0.765864
22     0.534056
23     0.683547
24     0.721389
25     0.693708
26     0.670416
27     0.629251
28     0.768977
29     0.694602
         ...   
571    0.575736
572    0.984579
573    0.429754
574    0.769001
575    0.921043
576    0.903164
577    0.543535
578    0.576542
579    0.838408
580    0.805498
581    0.439971
582    0.685980
583    0.698297
584    0.739760
585    0.550116
586    0.635176
587    0.251275
588    0.168720
589    0.732249
590    0.610444
591    0.908638
592    0.606007
593    0.652107
594    0.489042
595    0.480574
596    0.977626
597    0.882111
598    0.439166
599    0.650792
600    0.899007
Name: (5Z)-7-Oxozeaenol,

In [7]:
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=None)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)
    y_p = iraps.predict_proba(X_test)
    y_t = y_test < iraps.discretize_value
    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

AUC: 0.877358
AUC: 0.853365
AUC: 0.782931
AUC: 0.710000
AUC: 0.862534
AUC: 0.789091
AUC: 0.626841
AUC: 0.820000
AUC: 0.764706
AUC: 0.617251
Mean: 0.770408


## drug  Olaparib_(1)

In [ ]:
rna = pd.read_csv('./drug_respond/Olaparib_(1).tsv', sep='\t')
rna

In [ ]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=None)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)
    y_p = iraps.predict_proba(X_test)
    y_t = y_test < iraps.discretize_value
    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

### gridsearch

In [ ]:
from sklearn.metrics.scorer import _BaseScorer
class IRAPSScorer(_BaseScorer):
    def __call__(self, clf, X, y, sample_weight=None):
        y_true = y < clf.discretize_value
        y_pred = clf.predict_proba(X)
        if sample_weight is not None:
            return self._sign * self._score_func(y_true, y_pred,
                                                 sample_weight=sample_weight,
                                                 **self._kwargs)
        else:
            return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
    
    def _factory_args(self):
        return ", needs_iraps=True"

    
iraps_auc_scorer = IRAPSScorer(roc_auc_score, 1, {})

In [ ]:
roc_auc_scorer

In [ ]:
iraps_auc_scorer

In [ ]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=None)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)

    auc = iraps_auc_scorer(iraps, X_test, y_test)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

In [ ]:
cv = KFold(10, shuffle=True, random_state=0)

iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
kbest = SelectKBest(k=45000)
pip = Pipeline([('prep_1', kbest), ('estimator', iraps)])
search = GridSearchCV(pip, {}, cv = cv, scoring=iraps_auc_scorer, refit=False, verbose=10, return_train_score=True)


In [ ]:
pip.get_params()

In [ ]:
search.fit(X, y)
search.cv_results_

In [ ]:
help(G)

In [ ]:
cv = KFold(10, shuffle=True, random_state=0)


iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
search = IRAPSGirdSearchCV(iraps, {}, cv = cv, refit=False, verbose=10, return_train_score=False)
search.fit(X, y)
search.cv_results_


In [ ]:
estimator = XGBRegressor(n_estimator=10)
kbest = SelectKBest(k=100)
pip = Pipeline([('prep_1', kbest), ('estimator', estimator)])
search1 = GridSearchCV(pip, {})
search1.fit(X, y)
search1.cv_results_


In [ ]:
import sys
sys.modules['sklearn.model_selection._search']

In [ ]:
help(roc_auc_scorer)